# Sign Language Detection - Model Training
This notebook trains a CNN model to classify sign language gestures using image data.

In [ ]:
# 📦 Import necessary libraries
import os
import numpy as np
import matplotlib.pyplot as plt
import cv2
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical
import seaborn as sns

In [ ]:
# 📁 Load and preprocess dataset
DATASET_DIR = '/path/to/asl_alphabet_train'  # Replace this with your path
img_size = 64
categories = os.listdir(DATASET_DIR)
data = []
labels = []
for i, category in enumerate(categories):
    folder_path = os.path.join(DATASET_DIR, category)
    for img_name in os.listdir(folder_path)[:100]:
        img_path = os.path.join(folder_path, img_name)
        img = cv2.imread(img_path)
        img = cv2.resize(img, (img_size, img_size))
        data.append(img)
        labels.append(i)
data = np.array(data) / 255.0
labels = to_categorical(labels, num_classes=len(categories))

In [ ]:
# 🔀 Split dataset
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

In [ ]:
# 🧠 Build CNN model
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(img_size, img_size, 3)),
    MaxPooling2D(2,2),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(len(categories), activation='softmax')
])
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

In [ ]:
# 🚀 Train the model
history = model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

In [ ]:
# 📊 Evaluate model
loss, acc = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {acc*100:.2f}%")

In [ ]:
# 🧾 Classification report & confusion matrix
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test, axis=1)
print(classification_report(y_true, y_pred_classes))
cm = confusion_matrix(y_true, y_pred_classes)
plt.figure(figsize=(10,8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.title("Confusion Matrix")
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.show()

In [ ]:
# 💾 Save model and categories
model.save("sign_language_model.h5")
import joblib
joblib.dump(categories, "label_encoder.pkl")